## Problem Definition: We want: 
## A) To store key information generally pulled from mongo database in prepostRSA.ipynb directly within its associated project folder
## B) to rewrite relevant prepostRSA.ipynb code such that it pulls said information from the project folder rather than the mongo database.

## 1. Setup: Import key packages, ignore warnings, establish database connection

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
import os
import json

In [32]:
# original behavioral data from database
import pymongo as pm
from pymongo import MongoClient
conn = MongoClient('localhost', 20809)

DBNAME = conn['during_morph_drawing_recognition']
COLNAME = DBNAME['fmri3.files']
coll=COLNAME
DATADIR = 'neurosketch_data_3'



## 2. Answer Key Question: how is the database used in the original prepostRSA file?

In [11]:
# how is this behavioral data used in the original prepostRSA file?
# sole function :
def extract_condition_by_voxel_run_mat(this_sub,run_num, roi):
    w = this_sub
    these = coll.find({'wID': w}).sort('trialNum')   
    versionNum = these[0]['versionNum']

    design = [i for i in mdtd if i['version'] == int(versionNum)] # find which axes belong to which condition
    trained = design[0]['trained']
    near = design[0]['near']
    far1 = design[0]['far1']
    far2 = design[0]['far2']

    Tep = getEndpoints(trained)
    Nep = getEndpoints(near)
    condorder = Tep + Nep

    slot1 = load_data_and_apply_mask(this_sub,run_num,roi,obj2cope[condorder[0]])
    slot2 = load_data_and_apply_mask(this_sub,run_num,roi,obj2cope[condorder[1]])
    slot3 = load_data_and_apply_mask(this_sub,run_num,roi,obj2cope[condorder[2]])
    slot4 = load_data_and_apply_mask(this_sub,run_num,roi,obj2cope[condorder[3]])
    return np.vstack((slot1,slot2,slot3,slot4))

The sole piece of information derived from the mongo database for any subject is these[0]['versionNum'] where 'these' refers to coll.find({'wID': w}).sort('trialNum') and col is defined in setup.

## 3. Create dictionary of every possible this_sub / these[0]['versionNum'] pairing.

In [12]:
## get list of subject directories
proj_dir = '/home/jefan/sketchloop02/'
contents_dir = os.listdir(proj_dir)

sub_dirs = []
for i in contents_dir:
    try:
        if i.split('_')[1]=='neurosketch':
            sub_dirs.append(i)
    except:
        pass

sub_dirs = sorted(sub_dirs)

# issue with 1207161
sub_dirs = [s for s in sub_dirs if s != '1207161_neurosketch']

print(sub_dirs)
print(str(len(sub_dirs)) + ' subjects')

['0110171_neurosketch', '0110172_neurosketch', '0111171_neurosketch', '0112171_neurosketch', '0112172_neurosketch', '0112173_neurosketch', '0113171_neurosketch', '0115172_neurosketch', '0115174_neurosketch', '0117171_neurosketch', '0118171_neurosketch', '0118172_neurosketch', '0119171_neurosketch', '0119172_neurosketch', '0119173_neurosketch', '0119174_neurosketch', '0120171_neurosketch', '0120172_neurosketch', '0120173_neurosketch', '0123171_neurosketch', '0123173_neurosketch', '0124171_neurosketch', '0125171_neurosketch', '0125172_neurosketch', '1121161_neurosketch', '1130161_neurosketch', '1201161_neurosketch', '1202161_neurosketch', '1203161_neurosketch', '1206161_neurosketch', '1206162_neurosketch', '1206163_neurosketch', '1207162_neurosketch']
33 subjects


In [15]:
# pair subject with versionNum using a dictionary
version_dict = {}

# scan through each subject
for s in sub_dirs:
    # find its version number
    these = coll.find({'wID': s}).sort('trialNum')   
    versionNum = these[0]['versionNum']
    
    # pair
    print(s, versionNum)
    version_dict[s] = versionNum

0110171_neurosketch 18
0110172_neurosketch 21
0111171_neurosketch 24
0112171_neurosketch 27
0112172_neurosketch 30
0112173_neurosketch 33
0113171_neurosketch 21
0115172_neurosketch 24
0115174_neurosketch 27
0117171_neurosketch 30
0118171_neurosketch 33
0118172_neurosketch 33
0119171_neurosketch 18
0119172_neurosketch 21
0119173_neurosketch 24
0119174_neurosketch 27
0120171_neurosketch 30
0120172_neurosketch 33
0120173_neurosketch 18
0123171_neurosketch 24
0123173_neurosketch 30
0124171_neurosketch 33
0125171_neurosketch 27
0125172_neurosketch 21
1121161_neurosketch 24
1130161_neurosketch 27
1201161_neurosketch 30
1202161_neurosketch 21
1203161_neurosketch 33
1206161_neurosketch 18
1206162_neurosketch 21
1206163_neurosketch 24
1207162_neurosketch 30


## 4. Store output as json file for ready use

In [17]:
with open('versionNums.json', 'w') as fp:
    json.dump(version_dict, fp)

## 5. Finally, modify code so that extract_condition_by_voxel_run_mat() so that it accesses the json file instead of a mongo database for relevant info.

In [30]:
# behavioral data from database
import json
with open('versionNums.json') as json_data:
    coll = json.load(json_data)
coll

{'0110171_neurosketch': '18',
 '0110172_neurosketch': '21',
 '0111171_neurosketch': '24',
 '0112171_neurosketch': '27',
 '0112172_neurosketch': '30',
 '0112173_neurosketch': '33',
 '0113171_neurosketch': '21',
 '0115172_neurosketch': '24',
 '0115174_neurosketch': '27',
 '0117171_neurosketch': '30',
 '0118171_neurosketch': '33',
 '0118172_neurosketch': '33',
 '0119171_neurosketch': '18',
 '0119172_neurosketch': '21',
 '0119173_neurosketch': '24',
 '0119174_neurosketch': '27',
 '0120171_neurosketch': '30',
 '0120172_neurosketch': '33',
 '0120173_neurosketch': '18',
 '0123171_neurosketch': '24',
 '0123173_neurosketch': '30',
 '0124171_neurosketch': '33',
 '0125171_neurosketch': '27',
 '0125172_neurosketch': '21',
 '1121161_neurosketch': '24',
 '1130161_neurosketch': '27',
 '1201161_neurosketch': '30',
 '1202161_neurosketch': '21',
 '1203161_neurosketch': '33',
 '1206161_neurosketch': '18',
 '1206162_neurosketch': '21',
 '1206163_neurosketch': '24',
 '1207162_neurosketch': '30'}

In [33]:
#also need to modify this
def extract_condition_by_voxel_run_mat(this_sub,run_num, roi):
    versionNum = coll[this_sub]

    design = [i for i in mdtd if i['version'] == int(versionNum)] # find which axes belong to which condition
    trained = design[0]['trained']
    near = design[0]['near']
    far1 = design[0]['far1']
    far2 = design[0]['far2']

    Tep = getEndpoints(trained)
    Nep = getEndpoints(near)
    condorder = Tep + Nep

    slot1 = load_data_and_apply_mask(this_sub,run_num,roi,obj2cope[condorder[0]])
    slot2 = load_data_and_apply_mask(this_sub,run_num,roi,obj2cope[condorder[1]])
    slot3 = load_data_and_apply_mask(this_sub,run_num,roi,obj2cope[condorder[2]])
    slot4 = load_data_and_apply_mask(this_sub,run_num,roi,obj2cope[condorder[3]])
    return np.vstack((slot1,slot2,slot3,slot4))

Has too many dependencies to test here; just check revised prepostRSA.ipynb code.